## Implementing Unigram Tokenization Technique

Compared `BPE` to `WordPiece`, `Unigram` works in the other direction: it starts from a big vocabulary and `removes` tokens from it until it reaches the desired vocabulary size.

There are several options to use to `build` that `base vocabulary`: we can take the `most common substrings` in pre-tokenized words, for instance, or apply `BPE` on the initial corpus with a large vocabulary size.

At each step of the `training`, the `Unigram algorithm` computes a `loss` over the corpus given the current vocabulary. Then, for each symbol in the vocabulary, the algorithm computes how much the overall loss would `increase` if the symbol was removed, and looks for the symbols that would increase it the least. Those symbols have a `lower effect` on the overall loss over the corpus, so in a sense they are “less needed” and are the best candidates for removal.

This is all a very `costly operation`, so we don’t just remove the single symbol associated with the lowest loss increase, but the $p$ ($p$ being a hyperparameter you can control, usually 10 or 20) percent of the symbols associated with the lowest loss increase. This process is then repeated until the vocabulary has reached the `desired size`.

## Tokenization algorithm

A `Unigram model` is a type of language model that considers each token to be `independent` of the tokens before it.

The `probability` of a given token is its frequency in the original corpus, divided by the sum of all frequencies of all tokens in the vocabulary.

Consider the corpus: $("hug", \; 10), \; ("pug", \; 5), \; ("pun", \; 12), \; ("bun", \; 4), \; ("hugs", \; 5)$

An example of a base vocabulary is $["h",\; "u",\; "g",\; "hu",\; "ug",\; "p",\; "pu",\; "n",\; "un",\; "b",\; "bu",\; "s",\; "hug",\; "gs",\; "ugs"]$

Then for example the probability of the word "pug" is: $P(["p", "u", "g"]) = P("p")\cdot P("u")\cdot P("g") = \cfrac{5}{210}\cdot \cfrac{36}{210} \cdot \cfrac{20}{210} = 0.000389$

But also: $P(["pu", "g"]) = P("pu")\cdot P("g") = \cfrac{5}{210} \cdot \cfrac{20}{210} = 0.0022676$ and also $["p", "ug"]$

In general, tokenizations with the `least tokens` possible will have the `highest probability` (because of that division by 210 repeated for each token), which corresponds to what we want intuitively: to split a word into the least number of tokens possible.

The tokenization of a word with the `Unigram model` is then the tokenization with the highest probability. In the example of "pug", here are the probabilities we would get for each possible segmentation:
$["p", "u", "g"] : 0.000389$

$["p", "ug"] : 0.0022676$

$["pu", "g"] : 0.0022676$

So, "pug" would be tokenized as ["p", "ug"] or ["pu", "g"], depending on which of those segmentations is encountered first.

In general it’s going to be a bit hard finding `all` possible segmentation for each word. Thankfully `Viterbi algorithm` does exactly that. Essentially, we can build a `graph` to detect the possible segmentations of a given word by saying there is a branch from character $a$ to character $b$ if the subword from $a$ to $b$ is in the vocabulary, and attribute to that branch the probability of the subword.

To find the `path` in that graph that is going to have the best score the Viterbi algorithm determines, for each position in the word, the segmentation with the best score that ends at that position. Since we go from the beginning to the end, that best score can be found by `looping` through all subwords ending at the current position and then using the best tokenization score from the position this subword begins at. Then, we just have to `unroll` the path taken to arrive at the end.

## Computing Loss

Now that we have seen how the tokenization works, we can dive a little more deeply into the `loss` used during training. At any given stage, this `loss` is computed by `tokenizing every word` in the corpus, using the current vocabulary and the `Unigram` model determined by the frequencies of each token in the corpus.

Each word in the corpus has a `score`, and the loss is the negative log likelihood of those scores, i.e., $\sum_{w \in corpus}freq(w) \cdot (-log(P(w)))$.

Let’s go back to our example with the following corpus: $("hug", \;10), \;("pug", \;5), \;("pun", \;12), \;("bun", \;4), \;("hugs", \;5)$.

The tokenization of each word with their respective scores is:
$"hug": \;["hug"] \;(score \;0.071428)$

$"pug":\; ["pu",\; "g"] \;(score \;0.007710)$

$"pun": \;["pu", \;"n"] \;(score \;0.006168)$

$"bun": \;["bu",\; "n"] \;(score\; 0.001451)$

$"hugs": \;["hug",\; "s"] \;(score\; 0.001701)$

So the loss is: $10 * (-log(0.071428)) + 5 * (-log(0.007710)) + 12 * (-log(0.006168)) + 4 * (-log(0.001451)) + 5 * (-log(0.001701)) = 169.8$.


Now we need to compute how `removing` each token `affects` the loss. This is rather tedious, so we’ll just do it for two tokens here.  In this (very) particular case, we had two `equivalent tokenizations` of all the words: as we saw earlier, for example, "pug" could be tokenized ["p", "ug"] with the same score. Thus, removing the "pu" token from the vocabulary will give the exact same loss.

On the other hand, removing "hug" will make the loss `worse`, because the tokenization of "hug" and "hugs" will become:
$"hug": \;["hu",\; "g"] \;(score\; 0.006802)$

$"hugs": \;["hu", \;"gs"] \;(score \;0.001701)$

These changes will cause the loss to rise by: $- 10 * (-log(0.071428)) + 10 * (-log(0.006802)) = 23.5$.

Therefore, the token "pu" will probably be removed from the vocabulary, but not "hug".

## Setting the Corpus

In [52]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

## Calculating the Frequency of words in the Corpus

In [53]:
## Removing punctuations
from string import punctuation

for i in range(len(corpus)):
    for p in punctuation:
        corpus[i] = ' '.join([sub_s.strip().lower() for sub_s in corpus[i].split(p)])

print(corpus)

['this is the hugging face course', 'this chapter is about tokenization', 'this section shows several tokenizer algorithms', 'hopefully you will be able to understand how they are trained and generate tokens']


In [54]:
from collections import defaultdict

word_freq = defaultdict(lambda : 0)

for text in corpus:
    for word in text.split(' '):
        word_freq[word] += 1

print(word_freq)

defaultdict(<function <lambda> at 0x000002026762FDC0>, {'this': 3, 'is': 2, 'the': 1, 'hugging': 1, 'face': 1, 'course': 1, 'chapter': 1, 'about': 1, 'tokenization': 1, 'section': 1, 'shows': 1, 'several': 1, 'tokenizer': 1, 'algorithms': 1, 'hopefully': 1, 'you': 1, 'will': 1, 'be': 1, 'able': 1, 'to': 1, 'understand': 1, 'how': 1, 'they': 1, 'are': 1, 'trained': 1, 'and': 1, 'generate': 1, 'tokens': 1})


## Creating the Basic Vocabulary

Will contain all character and most common substrings.

In [55]:
## Calculating Character and Sub-String Frequencies

char_freq = defaultdict(lambda : 0)
subs_freq = defaultdict(lambda : 0)

for word, freq in word_freq.items():
    for i in range(len(word)):
        char_freq[word[i]] += freq # updating chracter frequency

        for j in range(i+2, len(word) + 1): # we need to contain `len(word)` because `slice` does not contain `end index`.
            subs_freq[word[i:j]] += freq    # updating substring frequency

In [56]:
## Sorting the Frequencies for the Sub-String, in order to keep the most frequent ones

sorted_subs = sorted(subs_freq.items(), key=lambda x: x[1], reverse=True)

sorted_subs[:10]

[('th', 6),
 ('is', 5),
 ('er', 5),
 ('to', 4),
 ('en', 4),
 ('thi', 3),
 ('this', 3),
 ('hi', 3),
 ('his', 3),
 ('ou', 3)]

In [57]:
len(sorted_subs)

387

In [58]:
## Concatenate those two dictionaries and creating a Vocabulary of size: 300

token_freq = list(char_freq.items()) + sorted_subs[: 300 - len(char_freq)]

# Converting the List of Typles into a Dictionary
token_freq = dict(token_freq)

In [59]:
len(token_freq)

300

## Converting Frequencies into Probabilities

In [60]:
from math import log

total_sum = sum(token_freq.values())

## Creating the Model: Going to store the `log`` of those probabilities
model = {token: -log(freq / total_sum) for token, freq in token_freq.items()}

## Viterbi Algorithm

We will store one `dictionary` per position in the word (from 0 to its total length), with two `keys`: the *index of the start* of the *last* token in the best segmentation, and the *score* of the best segmentation.

Populating the list is done with just two loops: the `main loop` goes over each start position, and the `second loop` tries all substrings beginning at that start position. If the substring is in the vocabulary, we have a new segmentation of the word up until that end position, which we compare to what is in best_segmentations.

In [61]:
## Best Segmentations: each segmentation is a List of Dictionaries (start, score) that represent each character of the word.
##  `start`: it's the best segmentation of the word that starts from `start` and ends in that character
##  `score`: it's the score of the segmentation

def _best_segmentations(word, model):
    # Contains starting character index of the best word segmentations that ends in this particular index
    best_segmentations = [{"start": 0, "score": 0}] + [{"start": None, "score": None} for _ in range(len(word))] # adding one more element for efficiency

    for start_idx in range(len(word)):
        best_score_at_start = best_segmentations[start_idx]["score"] # initially is 0, but then: `{"start": start_idx, "score": score}`

        for end_idx in range(start_idx + 1, len(word) + 1):
            token = word[start_idx: end_idx]

            if (token in model) and (best_score_at_start is not None):
                score = model[token] + best_score_at_start

                # If we have found a better segmentation ending at `end_idx``, we update
                if (best_segmentations[end_idx]["score"] is None) or (best_segmentations[end_idx]["score"] > score):
                    best_segmentations[end_idx] = {"start": start_idx, "score": score}
    
    return best_segmentations

In [62]:
_best_segmentations("huggs", model)

[{'start': 0, 'score': 0},
 {'start': 0, 'score': 3.871201010907891},
 {'start': 0, 'score': 6.269096283706261},
 {'start': 0, 'score': 6.269096283706261},
 {'start': 0, 'score': 6.269096283706261},
 {'start': 4, 'score': 9.973243209950986}]

In [63]:
## Encoding a word base on the best segmentation we have found

def segment_word(word, model):
    best_segmentations = _best_segmentations(word, model) # contains the starting indexes of the best segmentation ending at each character position
    segmentation = best_segmentations[-1]                 # contains the index and the score of the best segmentation ending at the last character

    if segmentation["score"] is None: return (["[UNK]"], None) # If the mode cannot segment the word, due to OOV

    score = segmentation["score"] # the score of the best segmentation ending at the last `word` character
    start = segmentation["start"] # the starting index of the best segmentation ending at the last `word` character
    end = len(word)
    tokens = []

    # Calculating the best segmentation
    while start != 0:
        tokens.insert(0, word[start: end])
        start, end = best_segmentations[start]["start"], start # start -> the index of the best segmentation ending at `end` | end -> start, we have segment the word from end to start
    
    tokens.insert(0, word[start: end]) # adding the last segment to the list

    return (tokens, score)


In [64]:
segment_word("huggs", model)

(['hugg', 's'], 9.973243209950986)

## Creating a Function for Computing the Loss

In [65]:
def compute_loss(model):
    l = 0
    for word, freq in word_freq.items():
        _, word_loss = segment_word(word, model)
        l += (freq * word_loss)

    return l

In [66]:
compute_loss(model)

268.6900959655888

## Computing Scores after Deleting a Token

In [67]:
from copy import deepcopy


def compute_scores(model):
    scores = {}
    model_loss = compute_loss(model)

    for token in model.keys():
        if len(token) == 1: # preventing the model for deleting character-level tokens
            scores[token] = float("inf")

        else:
            model_without_token = deepcopy(model)
            model_without_token.pop(token) # removing the token from the model - Dictionary

            scores[token] = compute_loss(model_without_token) - model_loss

    return scores

This approach is very `inefficient`, so `SentencePiece` uses an approximation of the loss of the model without token X: instead of starting from scratch, it just `replaces` token X by its segmentation in the vocabulary that is left. This way, all the scores can be computed at once at the same time as the model loss.

In [68]:
scores = compute_scores(model)

print(scores["ll"])
print(scores["his"])

3.0704231661555355
0.0


## Creating Vocabulary of `n` tokens

In [69]:
def fit(model, p_rem, size):
    while len(model) > size:
        # Calculating scores and sorting them from lower to higher
        scores = sorted(compute_scores(model).items(), key=lambda x: x[1])

        # Removing the percentage specified of low-scored tokens
        for i in range(int(len(model) * p_rem)):
            token_freq.pop(scores[i][0])

        # Initializing the new model
        total_sum = sum([freq for token, freq in token_freq.items()])
        model = {token: -log(freq / total_sum) for token, freq in token_freq.items()}

In [70]:
fit(model, 0.1, 100)

## Creating a Function for Tokenizing a Sentance

In [71]:
def tokenize(text, model, pre_f):
    # Preprocessing the Text
    pre_text = pre_f(text)

    tokenized_text = [segment_word(w, model)[0] for w in pre_text.split()] # 0: for only getting the segmentation

    return sum(tokenized_text, []) # starting from [] and adding elements from the `tokenized_text`

In [72]:
def pre(text):
    for p in punctuation:
        text = ' '.join([sub_s.strip().lower() for sub_s in text.split(p)])
    
    return text

In [74]:
tokenize("This is the Hugging Face course. Huggs to everyone!!!", model, pre_f=pre)

['this',
 'is',
 'the',
 'hugging',
 'face',
 'course',
 'hugg',
 's',
 'to',
 'ever',
 'y',
 'on',
 'e']

In [46]:
token_freq["hugg"]

6.269096283706261